In [37]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

class MatrixFactorization:
    def __init__(self, num_users, num_items, num_factors=10, learning_rate=0.01, regularization=0.01, num_epochs=10):
        self.num_users = num_users
        self.num_items = num_items
        self.num_factors = num_factors
        self.learning_rate = learning_rate
        self.regularization = regularization
        self.num_epochs = num_epochs

    def fit(self, ratings):
        self.ratings = ratings
#         self.user_factors = np.random.normal(scale=1.0/self.num_factors, size=(int(self.num_users), self.num_factors))
#         self.item_factors = np.random.normal(scale=1.0/self.num_factors, size=(int(self.num_items), self.num_factors))
        
        self.user_factors = np.ones((int(self.num_users), self.num_factors))
        self.item_factors = np.ones((int(self.num_items), self.num_factors))

        for epoch in range(self.num_epochs):
            for user, item, rating in self.ratings:
                user = int(user)
                item = int(item)
                error = rating - self.predict(user, item)
                self.user_factors[user] += self.learning_rate * (error * self.item_factors[item] - self.regularization * self.user_factors[user])
                self.item_factors[item] += self.learning_rate * (error * self.user_factors[user] - self.regularization * self.item_factors[item])

    
    def predict(self, user, item):
        return np.dot(self.user_factors[user], self.item_factors[item])

    def recommend(self, user, top_n=5):
        user_ratings = self.ratings[self.ratings[:, 0] == user]
        user_items = user_ratings[:, 1]
        predictions = np.array([self.predict(user, item) for item in range(int(self.num_items))])
        mask = np.logical_not(np.isin(range(int(self.num_items)), user_items))
        predictions[mask] = -np.inf
        top_items = np.argsort(predictions)[::-1][:top_n]
        return top_items

    def evaluate(self, test_ratings):
        test_users = test_ratings[:, 0]
        test_items = test_ratings[:, 1]
        test_ratings = test_ratings[:, 2]
        predicted_ratings = np.array([self.predict(user, item) for user, item in zip(test_users, test_items)])
        mae = mean_absolute_error(test_ratings, predicted_ratings)
        return mae

def recommendations(providers, userid, top_n):
    
    recommendations = mf.recommend(userid, top_n)
#     print("Recommendations for user", userid, ":", recommendations)
    
    recommend = pd.DataFrame(columns=providers.columns[1:])
    k = 0
    for el in recommendations.tolist():
        recommend.loc[k] = providers.iloc[el, ]
        k += 1
    return recommend

In [38]:
ratings = pd.read_csv('ratings.csv').drop(['Unnamed: 0'], axis=1)
providers = pd.read_csv('providers.csv').drop(['Unnamed: 0'], axis=1)
r = ratings.to_numpy()

num_users = np.max(r[:, 0]) + 1
num_items = np.max(r[:, 1]) + 1

mf = MatrixFactorization(num_users, num_items, num_factors=10, learning_rate=0.01, regularization=0.01, num_epochs=100)
mf.fit(r)

In [39]:
import sys
import os

def user2userPredictions(userid, pred_path):
    """
    Сделаем предикт для каждого пользователя и сохраним в файл prediction.csv
    
    :param
        - userid : пользователя id
        - pred_path : куда сохраняем
    """    
    # поиск поставщиков
    reg_num = set(ratings['Registration number'].tolist())
    user = set(ratings[ratings['User_id'] == userid]['Registration number'].tolist())
    diff = list(reg_num - user) 
    
    try:

        # цикл по всем выбраным пользователям для предикта
        for itemid in diff:
            
            # предикт для пользователя, по элементам
            r_hat = mf.predict(userid, itemid)  
#             print(r_hat)

            # сохраним
            with open(pred_path, 'a+') as file:
                line = '{},{},{}\n'.format(userid, itemid, r_hat)
                file.write(line)
                
    except IndexError:
        pass

def user2userMF():
    """
    Предикт для всех пользователей, даже с 1 рейтингом   
    """
    # список всех пользователей
    users = ratings['User_id'].unique()
    
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
    
    saved_predictions = 'predictionsMf.csv'    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    with open(saved_predictions, 'a+') as file:
        line = '{},{},{}\n'.format('User_id', 'Registration number', 'predicted_rating')
        file.write(line)
    
    for count, userid in enumerate(users):        
        # делаем предикт
        user2userPredictions(userid, saved_predictions)
        _progress(count)

def user2userRecommendation(userid, N=len(ratings.columns)):
    """
    Делаем предикт для пользователя
    """
    
    saved_predictions = 'predictionsMf.csv'
    
    predictions = pd.read_csv(saved_predictions, sep=',')
    
    predictions = predictions[predictions['User_id']==userid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)[:N]
    
    List = pd.merge(List, providers, on='Registration number', how='inner')
    
    return List

In [40]:
user2userMF()

Rating predictions. Progress status : 96.0%

In [41]:
user2userRecommendation(0, 5).drop(['Registration number', 'Предмет поставки', 'Важная информация'], axis=1)

,User_id,predicted_rating,Регистрационный номер,Наименование,Вид деятельности/отрасль,Телефон,Сводный индикатор,"Уставный капитал, RUB",Руководитель - ФИО
0,0,1.920602,1176196042681,"АЛЕКС ФРЕШ, ООО",Торговля оптовая фруктами и овощами,+7 (991) 0855161\n+7 (928) 9001606\n+7 (928) 9...,Низкий риск,10000.0,Баранцев Сергей Александрович
1,0,1.920602,1149204004290,"СЕВТОРГ, ООО",Торговля оптовая напитками,+7 (978) 7167193,Средний риск,700000.0,Кумалагов Заурбек Юрьевич
2,0,1.920602,1121690071030,"ДЕМЕТРА ПОВОЛЖЬЕ, ООО","Торговля оптовая пищевыми продуктами, напиткам...",+7 (917) 2707077,Низкий риск,10000.0,Ибрагимов Ильгам Надирович
3,0,1.920602,1026301504240,"ТОК-ПРОДУКТ, ООО","Торговля оптовая прочими пищевыми продуктами, ...",+7 (846) 9920722\n+7 (84649) 15303\n+7 (927) 2...,Средний риск,10000.0,Косов Константин Алексеевич
4,0,1.920602,1037739627265,"КОРТЛАЙН, ООО",Торговля оптовая мясом и мясными продуктами,+7 (495) 4465695\n+7 (495) 4465698,Низкий риск,10008400.0,Топоровский Владимир Эльевич
